In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import os

In [3]:
absolute_folder_path = os.path.abspath(os.path.dirname('__file__'))
relative_folder_path = "Gambar/"
folder_path = os.path.join(absolute_folder_path, relative_folder_path)

In [ ]:
class gantryCrane:
    def __init__(
        self,
        mc,
        mt,
        bt,
        br,
        g,
        L1,
        R1,
        J1,
        b1,
        rp1,
        Ke1,
        Kt1,
        L2,
        R2,
        J2,
        b2,
        rp2,
        Ke2,
        Kt2,
    ):
        # Gantry
        self.mc = mc
        self.mt = mt
        self.bt = bt
        self.br = br
        self.g = g

        # Motor 1
        self.L1 = L1
        self.R1 = R1
        self.J1 = J1
        self.b1 = b1
        self.rp1 = rp1
        self.Ke1 = Ke1
        self.Kt1 = Kt1

        # Motor 2
        self.L2 = L2
        self.R2 = R2
        self.J2 = J2
        self.b2 = b2
        self.rp2 = rp2
        self.Ke2 = Ke2
        self.Kt2 = Kt2

        # State
        self.x_init = 0.0
        self.l_init = 0.0
        self.theta_init = 0.0

        self.x = [0.0]
        self.x_dot = [0.0]
        self.x_dot_dot = [0.0]
        self.l = [0.0]
        self.l_dot = [0.0]
        self.l_dot_dot = [0.0]
        self.theta = [0.0]
        self.theta_dot = [0.0]
        self.theta_dot_dot = [0.0]

        self.x_target = 0.0
        self.l_target = 0.0
        self.theta_target = 0.0

        self.q = [0.0]
        self.q_dot = [0.0]
        self.q_dot_dot = [0.0]
        self.q_dot_dot_dot = [0.0]

        # Control
        self.sliding_surface1 = [0.0]
        self.sliding_surface2 = [0.0]
        self.u1 = [0.0]
        self.u2 = [0.0]
        self.u_limit = 0.0

    def target(self, x, l, theta=0.0):
        self.x_target = x
        self.l_target = l
        self.theta_target = theta

    def initialize(self, x_init, l_init, theta_init=0.0, u_limit=-1.0):
        self.x_init = x_init
        self.l_init = l_init
        self.theta_init = theta_init
        self.u_limit = u_limit

    def matrix_A(self,theta):
        matrix_A_ = np.matrix([[0.0, 0.0], [0.0, 0.0]])
        matrix_A_[0, 0] = self.L1 * self.rp1 * (
            self.mt + self.mc * np.sin(theta) ** 2
        ) / self.Kt1 + self.J1 * self.L1 / (self.Kt1 * self.rp1)
        matrix_A_[0, 1] = - self.L1 * self.mc * self.rp1 * np.sin(theta) / self.Kt1
        matrix_A_[1, 0] = - self.L2 * self.mc * self.rp2 * np.sin(theta) / self.Kt2
        matrix_A_[1, 1] = self.L2 * self.mc * self.rp2 / self.Kt2 + self.J2 * self.L2 / (self.Kt2 * self.rp2)
        return matrix_A_

    def matrix_B(self, theta, theta_dot):
        matrix_B_ = np.matrix([[0.0, 0.0], [0.0, 0.0]])
        matrix_B_[0, 0] = (
            + self.L1 * self.mc * self.rp1 * np.sin(2 * theta) * theta_dot / self.Kt1
            + self.R1 * self.rp1 * (self.mt + self.mc * np.sin(theta) ** 2) / self.Kt1
            + self.L1 * self.bt * self.rp1 / self.Kt1
            + self.J1 * self.R1 / (self.Kt1 * self.rp1)
            + self.L1 * self.b1 / (self.Kt1 * self.rp1)
        )
        matrix_B_[0, 1] = (
            - self.L1 * self.mc * self.rp1 * np.cos(theta) * theta_dot / self.Kt1
            - self.R1 * self.mc * self.rp1 * np.sin(theta) / self.Kt1
        )
        matrix_B_[1, 0] = (
            - self.L2 * self.mc * self.rp2 * np.cos(theta) * theta_dot / self.Kt2
            - self.R2 * self.mc * self.rp2 * np.sin(theta) / self.Kt2
        )
        matrix_B_[1, 1] = (
            + self.L2 * self.br * self.rp2 / self.Kt2
            + self.R2 * self.mc * self.rp2 / self.Kt2
            + self.J2 * self.R2 / (self.Kt2 * self.rp2)
            + self.L2 * self.b2 / (self.Kt2 * self.rp2)
        )
        return matrix_B_

    def matrix_C(self):
        matrix_C_ = np.matrix([[0.0, 0.0], [0.0, 0.0]])
        matrix_C_[0, 0] = self.R1 * self.bt * self.rp1 / self.Kt1 + self.Ke1 / self.rp1 + self.R1 * self.b1 / (self.Kt1 * self.rp1)
        matrix_C_[0, 1] = 0.0
        matrix_C_[1, 0] = 0.0
        matrix_C_[1, 1] = self.R2 * self.br * self.rp2 / self.Kt2 + self.Ke2 / self.rp2 + self.R2 * self.b2 / (self.Kt2 * self.rp2)
        return matrix_C_

    def matrix_D(self, l, theta, theta_dot):
        matrix_D_ = np.matrix([[0.0], [0.0]])
        matrix_D_[0, 0] = (
            2 * self.L1 * self.mc * self.rp1 * l * np.sin(theta) * theta_dot / self.Kt1
        )
        matrix_D_[1, 0] = - 2 * self.L2 * self.mc * self.rp2 * l * theta_dot / self.Kt2
        return matrix_D_

    def matrix_E(self, l, l_dot, theta, theta_dot):
        matrix_E_ = np.matrix([[0.0], [0.0]])
        matrix_E_[0, 0] = (
            self.L1 * self.mc * self.rp1 * l * np.cos(theta) * theta_dot ** 2 / self.Kt1
            + self.L1 * self.g * self.mc * self.rp1 * np.cos(2 * theta) / self.Kt1
            + self.L1 * self.mc * self.rp1 * np.sin(theta) * l_dot * theta_dot / self.Kt1
            + self.R1 * self.mc * self.rp1 * l * np.sin(theta) * theta_dot / self.Kt1
        )
        matrix_E_[1, 0] = (
            + self.L2 * self.g * self.mc * self.rp2 * np.sin(theta) / self.Kt2
            - self.L2 * self.mc * self.rp2 * l_dot * theta_dot / self.Kt2
            - self.R2 * self.mc * self.rp2 * l * theta_dot / self.Kt2            
        )
        return matrix_E_

    def matrix_F(self, theta):
        matrix_F_ = np.matrix([[0.0], [0.0]])
        matrix_F_[0, 0] = self.R1 * self.g * self.mc * self.rp1 * np.sin(theta) * np.cos(theta) / self.Kt1
        matrix_F_[1, 0] = - self.R2 * self.g * self.mc * self.rp2 * np.cos(theta) / self.Kt2
        return matrix_F_

    def update(self, l, l_dot, theta, theta_dot):
        control = (
            np.matmul(
                (self.matrix_B(theta, theta_dot) - np.matmul(self.matrix_A(theta), self.matrix_beta_)),
                self.q_dot_dot,
            )
            + np.matmul(
                (self.matrix_C() - np.matmul(self.matrix_A(theta), matrix_alpha_)),
                self.q_dot,
            )
            + self.matrix_D(theta, theta_dot) * self.theta_dot_dot
            + (self.matrix_E(l, l_dot, theta, theta_dot) - np.matmul(self.matrix_A(theta), matrix_lambda_))
            * self.theta_dot
            + self.matrix_F(theta)
            - self.k * self.sign_matrix(self.sliding_surface_now)
        )
        if (self.u_limit>0):
            self.u1.append(np.clip(control[0, 0], -self.u_limit, self.u_limit))
            self.u2.append(np.clip(control[1, 0], -self.u_limit, self.u_limit))

In [ ]:
# Physical Parameter
# Gantry
mc = 2.0
mt = 2.0
bt = 10.0
br = 10.0
g = 9.81

# Motor 1
L1 = 0.0015
R1 = 10.0
J1 = 0.0005
b1 = 0.0004
rp1 = 0.01
Ke1 = 0.05
Kt1 = 0.05

# Motor 2
L2 = 0.0025
R2 = 1.20
J2 = 0.0007
b2 = 0.0004
rp2 = 0.01
Ke2 = 0.08
Kt2 = 0.08

control_limit = 4.2  # Volt

In [ ]:
unconstrained = gantryCrane(mc, mt, bt, br, g, L1, R1, J1, b1, rp1, Ke1, Kt1, L2, R2, J2, b2, rp2, Ke2, Kt2)
unconstrained.initialize(0.0, 1.5, control_limit)
unconstrained.target(0.0, 0.0, 0.0)